# Decoerentes' solution to the first question of quantum chemistry

Decoerentes' team - Alisson, Bruno, Carlos, Gabriel, Miguel and Paulo

#Imports

In [ ]:
import sys
!{sys.executable} -m pip install numpy pandas matplotlib pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from pennylane import qchem

#Exercise 1.1

In [ ]:
def hamiltonian(n: int):
    """Defines the Hamiltonian K for a given number of qubits n."""
    # Use PBF lists with append in order gain time efficiency
    coeffs = []
    observables = []

    # The first term: (1/3) * sum_{i < j} X_i X_j
    for i in range(n):
        for j in range(i + 1, n):
            coeffs.append(1/3)
            observables.append(qml.PauliX(i) @ qml.PauliX(j))

    # The second term: -sum_{i=0}^{n-1} Z_i
    for i in range(n):
        coeffs.append(-1)
        observables.append(qml.PauliZ(i))

    return qml.Hamiltonian(coeffs, observables)

In [ ]:
def expectation_value_hamiltonian(n: int):
    """Simulates the circuit and returns the expectation value of K for ket PSI."""
    dev = qml.device("default.qubit", wires=n)

    @qml.qnode(dev)
    def circuit():
        # Prepare initial state |0⟩^(X)n, and apply Hadamard gates
        for i in range(n):
            qml.Hadamard(wires=i)

        # Return the expectation value of the Hamiltonian
        return qml.expval(hamiltonian(n))

    # Calculate and return the expectation value
    return circuit()

In [ ]:
n_qubits = 3
expectation_value = expectation_value_hamiltonian(n_qubits)
print(f"Expectation value of the Hamiltonian for {n_qubits} qubits: {expectation_value}")

Expectation value of the Hamiltonian for 3 qubits: 0.9999999999999991


#Exercise 1.2.a

In [ ]:
def ground_state_energy_h2(coords: list):
    """
    Function to compute the ground state energy of an H2 molecule using VQE.
    Takes a 6-element array representing the x, y, z coordinates of two hydrogen atoms.
    """
    symbols = ["H", "H"]
    # Use the input coordinates to define the molecule's geometry
    geometry = np.reshape(coords, (2, 3))  # Reshape the 6-element array into 2x3

    # Step 1: Define the molecule with qchem.molecule and set charge=0, multiplicity=1
    mol = qchem.Molecule(symbols, geometry, charge=0, mult=1, basis_name="sto-3g")

    # Step 2: Construct the electronic Hamiltonian using Jordan-Wigner transformation
    hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g")

    # Step 3: Define a device
    dev = qml.device("default.qubit", wires=qubits)

    # Step 4: Set up a variational quantum circuit for VQE
    def ansatz(params):
        qml.BasisState(np.array([1, 1]), wires=[0, 1])
        qml.RY(params[0], wires=0)
        qml.RX(params[1], wires=1)

    # Step 5: Use PennyLane's VQE solver
    opt = qml.GradientDescentOptimizer()
    params = np.random.random(2)  # Random initial parameters for the ansatz

    # Define the cost function for optimization
    @qml.qnode(dev)
    def cost_fn(params):
        ansatz(params)
        return qml.expval(hamiltonian)

    # Step 6: Perform optimization
    max_iterations = 1000
    for i in range(max_iterations):
        params, energy = opt.step_and_cost(cost_fn, params)

    return energy


In [ ]:
coords = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 1.0])  #Geometry for H2
energy = ground_state_energy_h2(coords)
print(f"Ground state energy: {energy} Hartrees")

Ground state energy: -1.0659096235252643 Hartrees


# Exercise 1.2.b

In [ ]:
def ising_hamiltonian(N: int, h: float):
    """Constructs the Ising Hamiltonian for N spins and field intensity h."""
    coeffs = []
    observables = []

    # First term: -sum Z_i Z_{i+1}
    for i in range(N - 1):
        coeffs.append(-1)
        observables.append(qml.PauliZ(i) @ qml.PauliZ(i + 1))

    # Second term: -h * sum X_i
    for i in range(N):
        coeffs.append(-h)
        observables.append(qml.PauliX(i))

    return qml.Hamiltonian(coeffs, observables)

In [ ]:
def vqe_spin_chain(N: int, h: float):
    """Function to compute ground state energy of spin chain using VQE."""
    dev = qml.device("default.qubit", wires=N)

    @qml.qnode(dev)
    def circuit(params: list):
        for i in range(N):
            qml.Hadamard(wires=i)
        qml.broadcast(qml.RY, wires=range(N), pattern="single", parameters=params)
        return qml.expval(ising_hamiltonian(N, h))

    # Initializing parameters for the variational ansatz
    params = np.random.random(N)
    opt = qml.GradientDescentOptimizer()

    # Optimization loop
    for i in range(100):
        params, energy = opt.step_and_cost(circuit, params)

    return energy

In [ ]:
N = 4
h = 1.0
energy = vqe_spin_chain(N, h)
print(f"Ground state energy for Ising chain with N={N}, h={h}: {energy}")

Ground state energy for Ising chain with N=4, h=1.0: -4.203742907267461


#Exercise 1.2.c

In [ ]:
def time_evolution_trotter(a, b, t, n):
    """Function to apply Trotterized time evolution to the initial state ket 00."""
    dev = qml.device('default.qubit', wires=2)

    def apply_h1(dt):
        """Applies the time evolution for H1 = a X0 X1."""
        # Controlled-X rotation (CNOT + rotations)
        qml.CNOT(wires=[0, 1])
        qml.RX(2 * a * dt, wires=1)
        qml.CNOT(wires=[0, 1])

    def apply_h2(dt):
        """Applies the time evolution for H2 = b Z0 Z1."""
        # Z0 Z1 is diagonal, so we can directly apply exp(-i b dt Z0 Z1)
        qml.CNOT(wires=[0, 1])
        qml.RZ(2 * b * dt, wires=1)
        qml.CNOT(wires=[0, 1])

    @qml.qnode(dev)
    def circuit():
        qml.BasisState(np.array([0, 0]), wires=[0, 1])

        # Perform Trotterized time evolution
        dt = t / n
        for _ in range(n):
            apply_h1(dt)
            apply_h2(dt)

        # Return the probabilities of the computational basis states
        return qml.probs(wires=[0, 1])

    # Execute the circuit
    return circuit()

In [ ]:
# Example parameters
a = 1.0  # Coefficient for X0 X1 term
b = 1.0  # Coefficient for Z0 Z1 term
t = 1.0  # Total time of evolution
n = 10   # Number of Trotter steps

# Call the function to simulate and get the probabilities
probs = time_evolution_trotter(a, b, t, n)
print(f"Probabilities of states |00⟩, |01⟩, |10⟩, |11⟩: {probs}")

Probabilities of states |00⟩, |01⟩, |10⟩, |11⟩: [0.50989939 0.49010061 0.         0.        ]
